In [256]:
# レビュー値ではなく売り上げ予測の方向にシフト
# とりあえず回帰から
import math
import pandas as pd
import numpy as np
import json
import ast
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import matplotlib.pyplot as plt

In [257]:
df1 = pd.read_csv("../movie/tmdb_5000_movies.csv")
df2 = pd.read_csv("../movie/tmdb_5000_credits.csv")
df3 = pd.merge(df1,df2,on="title")
df3.head()

#いらない場所を落とす
df3 = df3.drop(columns=['homepage', 'id',"original_title","overview","spoken_languages","status","tagline","vote_count","movie_id","popularity","original_language","production_countries"])
df3.head()
df3 = df3.dropna(axis=0,how="any")
# 収入が0のものを削除
df3 = df3.query("revenue >= 1000")
df3.shape

(3357, 11)

In [258]:
crews = df3["crew"]

crew_test = crews[1]
crew_test = ast.literal_eval(crew_test[1:-1])
for i in crew_test:
    print(i)

{'credit_id': '52fe4232c3a36847f800b579', 'department': 'Camera', 'gender': 2, 'id': 120, 'job': 'Director of Photography', 'name': 'Dariusz Wolski'}
{'credit_id': '52fe4232c3a36847f800b4fd', 'department': 'Directing', 'gender': 2, 'id': 1704, 'job': 'Director', 'name': 'Gore Verbinski'}
{'credit_id': '52fe4232c3a36847f800b54f', 'department': 'Production', 'gender': 2, 'id': 770, 'job': 'Producer', 'name': 'Jerry Bruckheimer'}
{'credit_id': '52fe4232c3a36847f800b503', 'department': 'Writing', 'gender': 2, 'id': 1705, 'job': 'Screenplay', 'name': 'Ted Elliott'}
{'credit_id': '52fe4232c3a36847f800b509', 'department': 'Writing', 'gender': 2, 'id': 1706, 'job': 'Screenplay', 'name': 'Terry Rossio'}
{'credit_id': '52fe4232c3a36847f800b57f', 'department': 'Editing', 'gender': 0, 'id': 1721, 'job': 'Editor', 'name': 'Stephen E. Rivkin'}
{'credit_id': '52fe4232c3a36847f800b585', 'department': 'Editing', 'gender': 2, 'id': 1722, 'job': 'Editor', 'name': 'Craig Wood'}
{'credit_id': '52fe4232c3a3

In [259]:
# 今回は監督を使いたいと思っているので、まずは映画の監督を抽出
crews_list = list()
num = 0
for each_crews in crews:
    num += 1
    if each_crews == "[]":
        crews_list.append("")
    else:
        each_crews = ast.literal_eval(each_crews[1:-1])
        producer_list = list()
        for each_crew in each_crews:
            try:
                if each_crew["job"] == "Director":
                    crew = each_crew["name"]
                    producer_list.append(crew)
            except:
                crew = each_crews["name"]
                producer_list.append(crew)
        crews_list.append(producer_list)
num

3357

In [260]:
#  ダブってるのがあるからなんとかする(重複解消)
for i in range(len(crews_list)):
    crews_list[i] = list(set(crews_list[i]))
len(crews_list)

3357

In [261]:
# それをもとにどれだけ出てきたかをカウント
crews_dict = dict()
for crews in crews_list:
    for director in crews:
        if director in crews_dict:
            crews_dict[director] += 1
        else:
            crews_dict[director] = 1
len(crews_dict) #監督の種類

1655

In [268]:
# さっき作った映画の辞書をもとに収入を入れていく
df4 = df3.filter(["revenue","crew"])
df4.head()

,revenue,crew
0,2787965087,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,961000000,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,880674609,"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,1084939099,"[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,284139100,"[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [263]:
# とりあえず辞書を作ってそこに格納
# ここに各々の監督の作品の数で売り上げを割って平均の売り上げを求める
# 配列の左が合計値,右が足した数
director_sales = crews_dict
for key in crews_dict:
    director_sales[key] = [0,0]
len(director_sales)


1655

In [264]:
sales = df4["revenue"]
for i in range(len(sales)):
    per_sales = sales.iloc[i]
    director = crews_list[i]
    for j in director:
        director_sales[j][0] += per_sales
        director_sales[j][1] += 1

In [265]:
# その後足した分だけ割っていく
new_director_sales = dict()
for key,value in director_sales.items():
    sum_sales = value[0]
    sum_num = value[1]
    new_director_sales[key] = sum_sales//sum_num
new_director_sales

{'James Cameron': 840509919,
 'Gore Verbinski': 453859051,
 'Sam Mendes': 387649829,
 'Christopher Nolan': 528435404,
 'Andrew Stanton': 527261338,
 'Sam Raimi': 391612682,
 'Nathan Greno': 591794936,
 'Byron Howard': 450887465,
 'Joss Whedon': 987943689,
 'David Yates': 742971665,
 'Zack Snyder': 353742483,
 'Bryan Singer': 351029375,
 'Marc Forster': 201298420,
 'Andrew Adamson': 521371889,
 'Rob Marshall': 356292276,
 'Barry Sonnenfeld': 313205533,
 'Peter Jackson': 722071424,
 'Marc Webb': 506218674,
 'Ridley Scott': 199347374,
 'Chris Weitz': 289660916,
 'Anthony Russo': 666167478,
 'Joe Russo': 666167478,
 'Peter Berg': 208970653,
 'Colin Trevorrow': 758768301,
 'Shane Black': 615612571,
 'Tim Burton': 238387017,
 'Brett Ratner': 220502662,
 'Dan Scanlon': 743559607,
 'Michael Bay': 486043719,
 'Joseph Kosinski': 343115667,
 'John Lasseter': 451203061,
 'Brad Lewis': 559852396,
 'Martin Campbell': 264906421,
 'Lee Unkrich': 1066969703,
 'McG': 191292042,
 'James Wan': 393719131,


In [266]:
# 最後に特徴量を作成
director_sum_sales = list()
for directors in crews_list:
    num = 0
    for i in range(len(directors)):
        director = directors[i]
        num += new_director_sales[director]
    num = num//(i+1)
    director_sum_sales.append(num)
director_sum_sales

[840509919,
 453859051,
 387649829,
 528435404,
 527261338,
 391612682,
 521341200,
 987943689,
 742971665,
 353742483,
 351029375,
 201298420,
 453859051,
 453859051,
 353742483,
 521371889,
 987943689,
 356292276,
 313205533,
 722071424,
 506218674,
 199347374,
 722071424,
 289660916,
 722071424,
 840509919,
 666167478,
 208970653,
 758768301,
 387649829,
 391612682,
 615612571,
 238387017,
 220502662,
 743559607,
 486043719,
 486043719,
 391612682,
 506218674,
 343115667,
 505527728,
 264906421,
 1066969703,
 191292042,
 393719131,
 201298420,
 351029375,
 377747922,
 351029375,
 181776654,
 306467823,
 181115642,
 486043719,
 338792339,
 331926147,
 485588203,
 376025927,
 527261338,
 220502662,
 369602630,
 276201692,
 291790424,
 742971665,
 521371889,
 351029375,
 528435404,
 718508837,
 416669114,
 403142372,
 115096235,
 313205533,
 150562222,
 182331619,
 189882125,
 191737236,
 155296362,
 248895105,
 718508837,
 403142372,
 403142372,
 396600000,
 758539785,
 301851401,
 15

In [267]:
len(director_sum_sales)

3357

In [271]:
# 作成した特徴量をデータフレームに格納
director_sales = pd.DataFrame(director_sum_sales,columns=["director_sales"])
director_sales

,director_sales
0,840509919
1,453859051
2,387649829
3,528435404
4,527261338
...,...
3352,76901
3353,6910344
3354,99000
3355,424760
